In [ ]:
##########################################WITH DATABASE###################################

import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime
from playsound import playsound
from numpy import nan
import pandas as pd
import numpy
import pymysql
from datetime import timedelta

def timecomp(s):
    number = int(s.split(':')[0]+s.split(':')[1]+s.split(':')[2])
    return number

count=0
vcount=0
current_in={}     #Employees currently in office
entered = 0       #Variable used to acknowledge the updated file when 1
current = 0



db = pymysql.connect(host="localhost",port=3306,user='root',password="redhat",db='py_mysql' )
with db.cursor() as cur:

        cur.execute("select * from encodings")

        version = cur.fetchall()
        db.commit()
        cur.close()

Names = []
encodingsKnown = []
for i in version:
    Names.append(i[0]) 
    encodingsKnown.append(i[1])
encodeKnown =[]
for i in encodingsKnown:
    
    encodeKnown.append(np.array([float(j) for j in i.replace('[','').replace(']','').replace(',','').split()]))
cap = cv2.VideoCapture(0)
while True:
    if current==0:
        #zp=zip([name],[in_time])
        db = pymysql.connect(host="localhost",port=3306,user='root',password="redhat",db='py_mysql' )

        with db.cursor() as cur:
            
            cur.execute("select * from current_in")

            version = cur.fetchall()
            db.commit()
            cur.close()
        current_in={}
        #print(version[1][0])
        if len(version)>0:
            for i in range(len(version)):
                #print(current[version[i][0]])
                current_in[version[i][0]]=version[i][1]
                current=1

    _, img = cap.read()
    image = cv2.resize(img,(0,0),None,0.25,0.25)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    facesInFrame = face_recognition.face_locations(image)
    encodesInFrame = face_recognition.face_encodings(image,facesInFrame)

    for encFace,faceLoc in zip(encodesInFrame,facesInFrame):
        #print(encFace,faceLoc)
        matchList = face_recognition.compare_faces(encodeKnown,encFace)
       # print(matchList)
        faceDis = face_recognition.face_distance(encodeKnown,encFace)
        print(faceDis)
        match1 = min(faceDis)          #min Distance 
        #match = faceDis.index(match1)
        match = np.argmin(faceDis) 

        if match1<0.39:

            db = pymysql.connect(host="localhost",port=3306,user='root',password="redhat",db='py_mysql' )
            count+=1
            vcount=0
            if count>=3:
                name = Names[match].upper()
                try:
                    playsound('D:/Audio/%s.mp3'%(name))
                except:
                    print(name)
                playsound('D:/59089586.mp3')
                count = 0

                name = Names[match].upper()
                y,w,z,x = faceLoc
                y, w, z, x1 = y*4,w*4,z*4,x*4
                cv2.rectangle(img,(x,y),(w,z),(0,255,0),2)
                cv2.rectangle(img,(x,z-35),(w,z),(0,255,0),cv2.FILLED)
                cv2.putText(img,name,(x+6,z-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                print(name)

                curr = datetime.now()
                dt = curr.strftime('%d-%m-%Y')
                time = curr.strftime('%H:%M:%S')
                itime = timecomp(time)
                ############################LOGIN############################################
                if name not in current_in:
                    zp = zip([name],[dt],[time],[None],[None])
                    zp1 = zip([name],[dt])
                    current=0
                    with db.cursor() as cur:

                        
                        cur.executemany("select * from in_out where Name=%s and Date=%s",zip([name],[dt]))
                        version = cur.fetchall()

                        if len(version)==0:
                            cur.executemany("insert into current_in values(%s,%s)",zip([name],[time]))
                            cur.executemany("insert into in_out values(%s,%s,%s,%s,%s)",zp)
                        else:
                            cur.executemany("insert into current_in values(%s,%s)",zip([name],[time]))
                            cur.executemany("select logout_time from in_out where Name=%s and Date=%s",zp1)
                            version = cur.fetchall()
                            last_out = version[-1][-1]
                            if timecomp(last_out)+500<timecomp(time):
                                cur.executemany("insert into in_out values(%s,%s,%s,%s,%s)",zp)

                        db.commit()
                        cur.close()

                ################################Logout######################################
                else:
                    print(';;;;;;;;;;;;;;;;;;',current_in[name])
                    if timecomp(current_in[name])+500<itime:
                        
                    
                        current=0
                        t1 = timedelta(hours = int(current_in[name].split(':')[0]), minutes = int(current_in[name].split(':')[1]))
                        t2 = timedelta(hours = int(time.split(':')[0]), minutes = int(time.split(':')[1]))
                        worktime = t2-t1
                        work_time=timecomp(time)-timecomp(current_in[name])
                        zp = zip([name],[dt],[current_in[name]])
                        zp1 = zip([name],[dt],[current_in[name]],[time],[worktime])
                        z=zip([name])
                        if timecomp(current_in[name])+500<timecomp(time):
                            with db.cursor() as cur:
                                cur.executemany("delete from current_in where Name=%s",z)
                                db.commit()
                                cur.close()
                            with db.cursor() as cur:
                                cur.executemany("delete from in_out where Name=%s and Date=%s and login_time=%s",zp)
                                cur.executemany("insert into in_out values(%s,%s,%s,%s,%s)",zp1)
                                db.commit()
                                cur.close()
                                
        else:
            #################################VISITOR#####################################
            vcount+=1

            if vcount>=5:

                y,w,z,x = faceLoc
                y, w, z, x1 = y*4,w*4,z*4,x*4
                cv2.rectangle(img,(x,y),(w,z),(0,255,0),2)
                cv2.rectangle(img,(x,z-35),(w,z),(0,255,0),cv2.FILLED)
                cv2.putText(img,'visitor',(x+6,z-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                print('Visitor')
                vcount=0
                count=0
    
    cv2.imshow('Webcam',img)
    k = cv2.waitKey(1)
    if k == 27:         # wait for ESC key to exit
        break
    #cv2.waitKey(1)
cv2.destroyAllWindows()